# Extras

Vamos a ver muy brevemente algunas herramientas más que podemos utilizar a la hora de acelerar nuestro código Python.

## f2py

f2py es una herramienta que crea un wrapper en Python de funciones escritas en Fortran. Se originó en el proyecto SciPy como forma de reutilizar código científico maduro ya existente y actualmente forma parte de NumPy.

* Guía de usuario: http://docs.scipy.org/doc/numpy/f2py/index.html
* Tutorial en Pybonacci: http://pybonacci.org/2013/02/22/integrar-fortran-con-python-usando-f2py/

![f2py](static/f2py.png)

(Fuente: www.ucs.cam.ac.uk/docs/course-notes/unix-courses/pythonfortran)

In [6]:
%%file vectores.f90
module vectores
 
    implicit none
 
    contains
 
    ! Producto escalar entre dos vectores u, v de longitud n
    function producto_escalar(n, u, v) result(p)
 
        integer, intent(in) :: n
        double precision, intent(in) :: u(n), v(n)
        double precision :: p
 
        p = dot_product(u, v)
 
    end function
 
    ! Producto vectorial entre dos vectores u, v de longitud 3
    function producto_vectorial(u, v) result(w)
 
        double precision, intent(in) :: u(3), v(3)
        double precision :: w(3)
 
        w(1) = u(2) * v(3) - u(3) * v(2)
        w(2) = u(3) * v(1) - u(1) * v(3)
        w(3) = u(1) * v(2) - u(2) * v(1)
 
    end function
 
end module

Writing vectores.f90


In [8]:
!f2py3 -c vectores.f90 -m vectores

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "vectores" sources
f2py options: []
f2py:> /tmp/tmpyf5wbz5o/src.linux-x86_64-3.4/vectoresmodule.c
creating /tmp/tmpyf5wbz5o/src.linux-x86_64-3.4
Reading fortran codes...
	Reading file 'vectores.f90' (format:free)
Post-processing...
	Block: vectores
			Block: vectores
				Block: producto_escalar
				Block: producto_vectorial
appenddecl: "dimension" not implemented.
Post-processing (stage 2)...
	Block: vectores
		Block: unknown_interface
			Block: vectores
				Block: producto_escalar
				Block: producto_vectorial
Building modules...
	Building module "vectores"...
		Constructing F90 module support for "vectores"...
		Creating wrapper for Fortran function "producto_escalar"("producto_escalar")...
			Constructing wrapper function

In [9]:
from vectores import *  # Importamos el módulo

u = np.array([1, 2, 3])
v = np.array([1, 0, -1])

vectores.producto_escalar(u, v)

-2.0

In [10]:
vectores.producto_vectorial(u, v)

array([-2.,  4., -2.])

In [11]:
vectores.producto_vectorial?

## bottleneck

bottleneck es un conjunto de funciones pensadas para manejar arrays de NumPy reescritas en Cython.

* Código fuente: https://github.com/kwgoodman/bottleneck
* Documentación: http://berkeleyanalytics.com/bottleneck/

Está especialmente pensado para arrays de datos donde hay valores vacíos, siendo muy apropiado para combinarlo con otras herramientas como pandas.

In [1]:
import numpy as np
arr = np.array([1, 2, np.nan, 4, 5])

In [2]:
import bottleneck as bn
bn.nanmean(arr)

3.0

In [13]:
%timeit np.nanmean(arr)

The slowest run took 5.15 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 65.1 µs per loop


In [14]:
%timeit bn.nanmean(arr)

The slowest run took 57.66 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 270 ns per loop


## numexpr

numexpr es una biblioteca para evaluar expresiones matemáticas que involucren arrays de NumPy de una forma muy rápida.

* Documentación: https://github.com/pydata/numexpr/wiki/Numexpr-Users-Guide

In [15]:
import numpy as np
import numexpr as ne

a = np.arange(10)
b = np.arange(0, 20, 2)

ne.evaluate("2*a+3*b")

array([ 0,  8, 16, 24, 32, 40, 48, 56, 64, 72], dtype=int64)